# Classifying Architectural Building Typologies Using Unsupervised Learning

## MLND Capstone
Jim Peraino
November 12th, 2017

## I. Definition

### Project Overview

In the domains of architecture and urban planning, building typology is an often referenced idea when analyzing neighbordhoods, buildings, and public spaces. Historically, buildings have been classified in two primary ways: by form (or shape), and by use (what happens inside). In the early 19th century, the architect Jean-Nicolas-Louis Durand proposed the notion of architectural typology in his book Precis des Lecons d'Architecture (1). Since then, many efforts have been made to apply machine learning techniques to Urban Design and Architectural analysis, including with software such as GIS (2) and spatial recognition systems for identifying ideal sites for new development projects (3).

As an architect myself, I'm curious about ways that we can quantify qualitative information about buildings, and vice versa. There are relatively few datasets in the field of architecture, and establishing and automating methods for classifying buildings, in particular into qualitative categories, will be essential in this endeavor. Being able to classify a building as a certain typology enables many other kinds of analysis. This information can be used to assess the characteristics of a neighborhood, predict property values, identify areas for new development, or countless other tasks related to the built environment.

### Problem Statement

Databases of building typologies do not exist for most of the world. To exacerbate this problem, buildings often no longer limit themselves to the typical typologies (schools, homes, hospitals, churches, for instance), and instead, are often mixed-use. However, physical charactistics of buildings can be easily observed or deduced from satelite images (3). 

__The problem, then, is: How can we automate the process of classifying buildings into their respective typologies by analyzing this physical information?__

The problem is quantifiable and measurable because we can logically assess any individual building as to the makeup of its use. It is a problem that is replicable anywhere on earth that there are buildings, since google earth shows satelite images from which we can mine physical data.

Unsupervised learning can be used on this task. As was used in the Customer Segment project, this solution can follow the process of: Data Exploration, Data Preprocessing, Feature Transformation, Clustering, and drawing conlusions(5). For more detail, see the project design section. 

New York City's Department of City Planning puts out a dataset called The Primary Land Use Tax Lot Output (PLUTO),  which contain information about plots of lands, the characteristics of the buildings on that land, and various administrative districts. This dataset was obtained as a download from Kaggle (4). This dataset contains information for over 65,000 lots. 

Since my goal is to use physical characteristics to classify buildings into typological groups, I will test different combinations of Physical Characteristics to see which best enable clustering of the data into building typologies. I will then compare the percentages of commercial/residential/office etc. floor areas in each cluster to better understand and evaluate the clusters.

### Metrics

The evaluation metric will be the extent to which each cluster is homogenous in terms of its program (residential, retail, etc). This can be determined by calculating the standard deviation of the primary programmatic element in  each cluster. A smaller standard deviation would indicate that the cluster is more homogeneous, and a larger standard deviation would indicate that it is heterogeneous. 



## II. Analysis

### Data Exploration

__Dataset Features__

While the dataset has roughly 80 variables for each lot, those of particular interest are:

__Physical Characteristics__ (Used to create the model)
* Total Building Floor Area
* Lot Area 
* Number of Floors
* Lot Frontage
* Lot Depth
* Building Frontage
* Building Depth
* Floor Area Ration (FAR)

__Other Data__ (Used as the benchmark)
* Commercial, Residential, Office, Retail, Factory, Storage, Garage, and Other Floor Area
* Building Class
* Zoning District

In [1]:
### SOURCE: Code is based on Udacity MLND Customer Segments Project
# Import libaries necessary for this project
import numpy as np
import pandas as pd
from IPython.display import display

# Import supplementary visualizations code visuals code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the PLUTO Buildings Dataset for Brooklyn
try: 
    data = pd.read_csv("BK.csv", 
            low_memory=False,
            # Dataset has 80+ features, so load only those which are relevant
            usecols=["Address", "LotArea", "BldgArea", "BldgFront", "BldgDepth", "NumFloors", "NumBldgs", "LotFront", "LotDepth", "ComArea", "ResArea", "OfficeArea", "RetailArea", "GarageArea", "StrgeArea", "FactryArea"])
    
    print "PLUTO Brooklyn Dataset has {} samples with {} features each.".format(*data.shape)
    
except:
    print "Dataset could not be loaded."

PLUTO Brooklyn Dataset has 277131 samples with 16 features each.


In [2]:
# Display a description of the dataset
display(data.describe())

,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,NumBldgs,NumFloors,LotFront,LotDepth,BldgFront,BldgDepth
count,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,2.771310e+05,277131.000000,277131.000000,277131.000000,277131.000000,277131.000000,277131.000000
mean,6.028453e+03,5.177061e+03,1.460424e+03,3.637264e+03,2.927963e+02,2.485272e+02,8.346268e+01,1.042189e+02,1.881247e+02,1.211665,2.296141,32.246961,100.567182,24.067890,49.746923
std,4.300028e+05,5.477153e+04,5.126787e+04,1.870863e+04,7.991499e+03,3.746235e+03,3.386011e+03,3.968310e+03,5.214383e+03,0.913857,1.200165,58.948294,49.369778,33.224805,35.316955
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.900000e+03,1.648000e+03,0.000000e+00,1.400000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,2.000000,20.000000,100.000000,18.000000,38.000000
50%,2.134000e+03,2.368000e+03,0.000000e+00,2.052000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,2.000000,22.000000,100.000000,20.000000,46.000000
75%,2.833000e+03,3.312000e+03,0.000000e+00,2.860000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,3.000000,28.920000,100.000000,22.000000,57.000000
max,2.073912e+08,2.400000e+07,2.400000e+07,1.800000e+06,1.239070e+06,1.263000e+06,1.285000e+06,1.118000e+06,1.324592e+06,236.000000,119.000000,9999.000000,7960.000000,6020.000000,7360.000000


__Dataset Statistics__
As noted in the table above, there is a large range in the physical characteristics in the data. The number of floors in the middle 50% of the data ranges from 2-3 floors, however the maximum number of floors is 119. This makes sense when thinking about a city composed mostly of small buildings but with several taller buildings near its downtown area. This trend persists throughout the building characteristics, with characteristics such as Lot Front and Lot Depth staying relatively consistently between 20 and 100' for more than 50% of the data, but with significant shifts at the upper end (with some lot depths more than a mile long). Tall buildings are not outliers in this case, since they are a distinct typology, they just occur less often, so we will need to be sure to pick samples that account for this. 

__Abnormalities__
The dataset includes all lots in Brooklyn, even those without buildings. These will need to be removed as outliers/irrelevant because including them results in several features being listed with values of zero (Number of Floors, Building Front, etc). Notably, large buildings should *not* be removed as outliers since they may correspond to a specific type of building. 

Empty lots are removed from the data below.

In [3]:
# Limit samples to sites with only one building
data = data[data.NumBldgs == 1]

# Remove samples with no data for physical characteristics
data = data[data.NumFloors != 0]
data = data[data.BldgFront != 0]
data = data[data.BldgDepth != 0]
data = data[data.LotFront != 0]
data = data[data.LotDepth != 0]

# Display a new description of the dataset
print "PLUTO Brooklyn Dataset has {} samples with {} features each.".format(*data.shape)
display(data.describe())

PLUTO Brooklyn Dataset has 201272 samples with 16 features each.


,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,NumBldgs,NumFloors,LotFront,LotDepth,BldgFront,BldgDepth
count,2.012720e+05,2.012720e+05,2.012720e+05,201272.000000,2.012720e+05,201272.000000,201272.000000,2.012720e+05,201272.000000,201272.0,201272.000000,201272.000000,201272.000000,201272.000000,201272.000000
mean,3.259272e+03,4.878184e+03,1.241802e+03,3538.244495,2.865332e+02,269.198175,75.493909,1.113859e+02,184.396245,1.0,2.439267,29.419780,98.330185,25.759029,52.860792
std,1.632369e+04,1.655823e+04,1.263097e+04,10401.211523,7.755732e+03,2586.980552,2140.159556,3.997484e+03,4078.444038,0.0,1.085375,33.055479,27.900130,34.438155,36.055657
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1.0,1.000000,0.020000,6.580000,1.000000,1.000000
25%,1.825000e+03,1.833000e+03,0.000000e+00,1512.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1.0,2.000000,20.000000,96.670000,19.000000,40.000000
50%,2.000000e+03,2.520000e+03,0.000000e+00,2120.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1.0,2.000000,20.000000,100.000000,20.000000,48.000000
75%,2.517000e+03,3.510000e+03,0.000000e+00,3000.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1.0,3.000000,25.670000,100.000000,22.000000,58.000000
max,4.810000e+06,1.420558e+06,1.330743e+06,685893.000000,1.239070e+06,500630.000000,278406.000000,1.118000e+06,581250.000000,1.0,119.000000,2696.000000,2036.000000,6020.000000,7360.000000


__Selecting Samples__

Several representative building types are identified to put real buildings to the data.

* Mid-sized Retail Building: 128 Court Street
* Large Office Building: 345 Adams Street
* Single Family Row House: 590 11 Street
* Large Apartment Building: 235 Gold Street

In [8]:
# Identify addresses for samples
addresses = ['128 COURT STREET', '345 ADAMS STREET', '590 11 STREET', '235 GOLD STREET']
indices = []

# Get index of each sample
for i in addresses:
    index = data[data.Address == i].index[0]
    indices.append(index)

# Create a DataFrame of the chosen samples
samples = pd.DataFrame(data.loc[indices], columns = data.keys()).reset_index(drop = True)
print "Chosen samples from PLUTO database:"
display(samples)



Chosen samples from PLUTO database:


,Address,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,NumBldgs,NumFloors,LotFront,LotDepth,BldgFront,BldgDepth
0,128 COURT STREET,14518,28150,28150,0,0,28150,0,0,0,1,1.0,138.31,100.00,65.00,137.50
1,345 ADAMS STREET,25056,297845,297845,0,262183,35662,0,0,0,1,14.0,261.75,97.87,261.00,97.87
2,590 11 STREET,1858,2526,0,1694,0,0,0,0,0,1,2.0,18.58,100.00,18.58,44.00
3,235 GOLD STREET,39930,326609,25000,301609,0,25000,0,0,0,1,12.0,315.25,93.58,315.00,90.00


In [ ]:
# Select indices of samples from dataset
indices = [15004]

samples = pd.DataFrame(data.loc[indices], columns = data.keys()).reset_index(drop = True)
print "Chosen samples from PLUTO dataset:"
display(samples)

### Exploratory Visualization

TODO

### Algorithms and Techniques

TODO

### Benchmark

Existing methods for classifying building typology enter the use of each building one by one, through analysis of the use. In the case of this dataset, this data is included, and will serve as the benchmark model. This data will not be used in constructing the new model, but will instead be used to compare its performance. The benchmark model should have near 100% accuracy, as each item is entered individually. We expect that the unsupervised learning model will have significantly lower accuracy than the benchmark since it will be using only the information that can be gained from physical analysis. 

## III. Methodology

### Data Preprocessing

TODO

### Implementation

TODO

### Refinement

TODO

## IV. Results

### Model Evaluation and Validation

TODO

### Justification

## V. Conclusion

### Free-Form Visualization

TODO

### Reflection

TODO

### Improvement

TODO

### References:

(1) https://books.google.com/books?hl=en&lr=&id=wilTAAAAcAAJ&oi=fnd&pg=PA1&dq=precis+durand&ots=hY1zElBwh2&sig=clePUvTcCCBGFLNlBVFp0j4XPJw#v=onepage&q=precis%20durand&f=false

(2) https://pdfs.semanticscholar.org/24c5/4f200302943cde042aee677956adc0b2498e.pdf

(3) http://certainmeasures.com/spatial_recognition.html

(4) https://www.kaggle.com/new-york-city/nyc-buildings

(5) Udacity MLND Customer Segment Project